# Classificação de Células de Leucemia com Redes Neurais

**Autor:** Pietro Esteves

## Introdução

Este projeto foi desenvolvido como parte do meu Trabalho de Conclusão de Curso (TCC) e tem como objetivo a criação de um modelo de deep learning para a classificação de diferentes tipos de células sanguíneas, com foco na identificação de células de leucemia. A leucemia é um tipo de câncer que afeta as células do sangue e da medula óssea, e a sua detecção precoce é fundamental para o tratamento. O trabalho será publicado no Repositório da UFC em breve e assim que sair, colocarei um link aqui.

Utilizamos um conjunto de dados de imagens de células sanguíneas do Kaggle e a biblioteca `fastai` para treinar um modelo de rede neural convolucional (CNN) capaz de classificar as células em quatro categorias: 'Benigno', 'Inicial', 'Pré B' e 'Pró B'.

## 1. Importação das Bibliotecas

Nesta primeira etapa, importamos todos os módulos e bibliotecas necessários para o desenvolvimento do projeto. Utilizamos `os` para manipulação de caminhos de arquivos, `pandas` para a criação e manipulação de dataframes, e `fastai.vision` para as funcionalidades de visão computacional.

In [ ]:
import os
import pandas as pd
from fastai.vision.all import *
from fastai.vision import models
from fastai.metrics import error_rate, accuracy
import warnings

# Ignora avisos para uma saída mais limpa
warnings.filterwarnings("ignore")

# Define uma semente para garantir a reprodutibilidade dos resultados
set_seed(42)

## 2. Carregamento e Preparação dos Dados

Para este projeto, utilizamos um dataset público do Kaggle. A seguir, realizamos o download dos dados e os organizamos em um formato adequado para o treinamento do modelo.

In [ ]:
import kagglehub

# Realiza o download do dataset a partir do Kaggle Hub
path = kagglehub.dataset_download("mehradaria/leukemia")

print(f"Diretório dos dados: {path}")

### 2.1. Criação do DataFrame

Após o download, criamos um DataFrame do `pandas` contendo os caminhos das imagens e seus respectivos rótulos. Essa estrutura de dados facilita a manipulação e a organização das informações.

In [ ]:
# Define o diretório principal das imagens
data_dir = path + '/Original'
filepaths = []
labels = []

# Mapeia os rótulos originais em inglês para português
label_map = {
    'Early': 'Inicial',
    'PreB': 'Pré B',
    'ProB': 'Pró B',
    'Benign': 'Benigno'
}

# Itera sobre as pastas de cada classe de célula
for fold in os.listdir(data_dir):
    foldpath = os.path.join(data_dir, fold)
    if os.path.isdir(foldpath):
        for file in os.listdir(foldpath):
            fpath = os.path.join(foldpath, file)
            # Adiciona o caminho relativo do arquivo à lista
            filepaths.append(os.path.relpath(fpath, data_dir))
            # Adiciona o rótulo traduzido à lista
            translated_label = label_map.get(fold, fold)
            labels.append(translated_label)

# Cria o DataFrame com os caminhos e os rótulos
Fseries = pd.Series(filepaths, name='filepaths')
Lseries = pd.Series(labels, name='labels')
df = pd.concat([Fseries, Lseries], axis=1)

### 2.2. Análise da Distribuição das Classes

É uma boa prática verificar a distribuição das classes no conjunto de dados. Um desbalanceamento significativo pode impactar o desempenho do modelo. O gráfico abaixo mostra a quantidade de imagens para cada tipo de célula.

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

# Plota a distribuição das classes
df.groupby('labels').size().plot(kind='barh', color=sns.palettes.mpl_palette('tab10'))
plt.gca().spines[['top', 'right',]].set_visible(False)
plt.ylabel('Tipos de célula')
plt.xlabel('Número de Imagens')
plt.title('Distribuição das Classes de Células')
plt.show()

### 2.3. Preparação dos DataLoaders

Com o DataFrame criado, utilizamos a classe `ImageDataLoaders` do `fastai` para preparar os dados para o treinamento. Esta classe lida com a divisão dos dados em conjuntos de treino e validação, além de aplicar transformações às imagens, como o redimensionamento para um tamanho padrão (224x224 pixels).

In [ ]:
dls = ImageDataLoaders.from_df(df,
                                fn_col=0, 
                                label_col=1, 
                                valid_pct=0.2, # 20% dos dados para validação
                                folder=data_dir,
                                item_tfms=Resize(224))


### 2.4. Visualização de um Lote de Dados

Para verificar se os dados foram carregados corretamente, exibimos um lote de imagens com seus respectivos rótulos.

In [ ]:
dls.show_batch(max_n=16)

## 3. Treinamento do Modelo

Nesta seção, definimos e treinamos o nosso modelo de rede neural.

### 3.1. Escolha do Modelo

Decidimos utilizar a arquitetura **ResNet-34**, uma rede neural convolucional pré-treinada no dataset ImageNet. O uso de um modelo pré-treinado (transfer learning) é uma técnica eficaz que nos permite aproveitar o conhecimento aprendido em um grande conjunto de dados para o nosso problema específico, acelerando o treinamento e melhorando a performance.

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)

### 3.2. Encontrando a Taxa de Aprendizagem Ideal

A taxa de aprendizagem (learning rate) é um hiperparâmetro crucial no treinamento de redes neurais. Uma taxa muito alta pode fazer com que o modelo não convirja, enquanto uma taxa muito baixa pode tornar o treinamento excessivamente lento. A função `lr_find()` do `fastai` nos ajuda a encontrar uma taxa de aprendizagem ótima, que é tipicamente um ponto antes do valor mínimo da curva de perda.

In [ ]:
learn.lr_find()

### 3.3. Treinamento com `fine_tune`

A função `fine_tune` do `fastai` é uma maneira eficiente de aplicar o transfer learning. Ela primeiro treina as camadas finais do modelo (a "cabeça" da rede) e depois ajusta o restante da rede com uma taxa de aprendizagem menor. Isso permite que o modelo se adapte ao nosso dataset específico sem perder o conhecimento adquirido no treinamento inicial.

In [ ]:
# Realiza o fine-tuning do modelo por 4 épocas
learn.fine_tune(4, 0.001)

## 4. Análise dos Resultados

Após o treinamento, é importante analisar o desempenho do modelo para entender onde ele está acertando e onde está errando.

### 4.1. Matriz de Confusão

A matriz de confusão é uma excelente ferramenta para visualizar o desempenho do modelo. Ela mostra o número de predições corretas e incorretas para cada classe. A diagonal principal representa as classificações corretas.

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

### 4.2. Análise das Maiores Perdas

A função `plot_top_losses` nos mostra as imagens em que o modelo teve mais dificuldade para classificar (maiores valores de perda). Isso é útil para identificar possíveis problemas nos dados, como imagens mal rotuladas ou de baixa qualidade.

In [ ]:
interp.plot_top_losses(9, figsize=(15,10))

## 5. Testando o Modelo com uma Imagem

Finalmente, podemos testar o modelo com uma imagem específica para ver a sua predição em um exemplo prático.

In [ ]:
# Carrega uma imagem de teste
img = PILImage.create(path + '/Original/Benign/Im110_0.jpg')
# Realiza a predição
pred,_,_ = learn.predict(img)
print(f'A predição é: {pred}')

## Conclusão

Este projeto demonstrou a eficácia do uso de redes neurais convolucionais e transfer learning para a classificação de células de leucemia. O modelo treinado alcançou uma alta acurácia, mostrando o potencial dessa abordagem para auxiliar no diagnóstico médico.

Como próximos passos, poderíamos explorar outras arquiteturas de CNN, aumentar o conjunto de dados com técnicas de data augmentation mais avançadas, de interpretabilidade como o GradCAM, ou até mesmo desenvolver uma aplicação web para utilizar o modelo de forma interativa.